# Github

In [9]:
!git clone https://github.com/cyprien-n/Cloud-_Data_Structure.git

Cloning into 'Cloud-_Data_Structure'...


In [5]:
cd Cloud-_Data_Structure

c:\Users\Cyprien\OneDrive - De Vinci\Documents\ESILV\A5\Structure_de_donnee\Cloud-_Data_Structure


In [4]:
!git pull

Updating 0a71d6e..57c5710
Fast-forward
 maintest.py | 224 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
 1 file changed, 224 insertions(+)
 create mode 100644 maintest.py


From https://github.com/cyprien-n/Cloud-_Data_Structure
   0a71d6e..57c5710  main       -> origin/main


In [24]:
!git add .
!git commit -m "Adding Cyprien'Notebook"
!git push origin main

[main 4c98a36] Adding Cyprien'Notebook
 1 file changed, 24 insertions(+), 14 deletions(-)


To https://github.com/cyprien-n/Cloud-_Data_Structure.git
   828cce4..4c98a36  main -> main


# Project 

In [1]:
pip install pymongo

     -------------------------------------- 472.6/472.6 KB 1.8 MB/s eta 0:00:00
     -------------------------------------- 300.4/300.4 KB 1.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Cyprien\anaconda3\python.exe -m pip install --upgrade pip' command.


## End-User Queries

### EU1

In [3]:
from pymongo import MongoClient
import time

# Replace these values with your MongoDB connection details
mongo_host = 'localhost'
mongo_port = 27017
mongo_database = 'Walmart_cloud'
collection_name = 'sales'

# Number of times to execute the pipeline for averaging
num_iterations = 10

# Vary the number of shards from 1 to 6
for num_shards in range(1, 7):
    # Connect to MongoDB
    client = MongoClient(f'mongodb://{mongo_host}:{mongo_port}/')
    db = client[mongo_database]

    # Access the sharded collection
    sales = db[collection_name]

    # Define the aggregation pipeline
    pipeline = [
        {
            '$match': {
                'store_nbr': 1, 
                'item_nbr': 9
            }
        },
        {
            '$group': {
                '_id': None, 
                'total_units': {
                    '$sum': '$units'
                }
            }
        },
        {
            '$project': {
                '_id': 0, 
                'total_units': 1
            }
        }
    ]

    execution_times = []

    # Execute the pipeline multiple times and record the execution times
    for i in range(num_iterations):
        start_time = time.time()
        result = list(collection.aggregate(pipeline))
        end_time = time.time()
        execution_time = end_time - start_time
        execution_times.append(execution_time)

        # Print the result of the first iteration (for verification purposes)
        if i == 0:
            print(f"Query Result with {num_shards} shards:", result)

    # Remove the minimum and maximum execution times
    execution_times = sorted(execution_times)[1:-1]

    # Calculate and print the average execution time
    average_execution_time = sum(execution_times) / len(execution_times)
    print(f"Average Execution Time with {num_shards} shards: {average_execution_time} seconds")

    # Close the connection
    client.close()


Query Result with 1 shards: [{'total_units': 27396}]


KeyboardInterrupt: 

#### Explication de la requête EU1

In analyzing the query performance across different sharding configurations, the following key points can be highlighted:

1. **Consistent Query Results:**
   - The query consistently returns the same total_units value across varying sharding configurations, indicating accurate and consistent results.

2. **Marginal Execution Time Variations:**
   - The average execution time experiences marginal variations with an increase in the number of shards.
   - Execution times range from approximately 1.14 to 1.20 seconds, suggesting that the impact of sharding on this specific query is relatively small.

3. **Performance Evaluation:**
   - The observed variations in execution times are likely influenced by factors such as network latency, hardware capabilities, and the complexity of the sharding configuration.
   - MongoDB's automatic parallelization of queries across shards appears to contribute to similar performance metrics for this query.

4. **Considerations for Optimization:**
   - While the current query performs well across different sharding configurations, ongoing monitoring and analysis may reveal opportunities for optimization.
   - Evaluating the sharding key selection and considering other query patterns can provide insights into potential performance improvements.

In summary, the query demonstrates consistent and satisfactory performance across varying sharding configurations. Ongoing assessment of query patterns and system resources will help fine-tune sharding strategies for optimal performance.

### EU2

In [ ]:
[
    {
        '$match': {
            'store_nbr': 12345
        }
    }, {
        '$sort': {
            'date': -1
        }
    }, {
        '$limit': 1
    }, {
        '$project': {
            '_id': 0, 
            'snowfall': '$snowfall', 
            'precipitation': '$precipitation'
        }
    }
]

In [8]:
from pymongo import MongoClient
import time

# Replace these values with your MongoDB connection details
mongo_host = 'localhost'
mongo_port = 27017
mongo_database = 'Walmart_cloud'
collection_name = 'sales'

# Number of times to execute the pipeline for averaging
num_iterations = 10

# Vary the number of shards from 1 to 6
for num_shards in range(1, 2):
    # Connect to MongoDB
    client = MongoClient(f'mongodb://{mongo_host}:{mongo_port}/')
    db = client[mongo_database]

    # Access the sharded collection
    collection = db[collection_name]

    # Define the aggregation pipeline
    pipeline = [
    {
        '$match': {
            'store_nbr': 12345
        }
    }, {
        '$sort': {
            'date': -1
        }
    }, {
        '$limit': 1
    }, {
        '$project': {
            '_id': 0, 
            'snowfall': '$snowfall', 
            'precipitation': '$precipitation'
        }
    }
]

    execution_times = []

    # Execute the pipeline multiple times and record the execution times
    for i in range(num_iterations):
        start_time = time.time()
        result = list(collection.aggregate(pipeline))
        end_time = time.time()
        execution_time = end_time - start_time
        execution_times.append(execution_time)

        # Print the result of the first iteration (for verification purposes)
        if i == 0:
            print(f"Query Result with {num_shards} shards:", result)

    # Remove the minimum and maximum execution times
    execution_times = sorted(execution_times)[1:-1]

    # Calculate and print the average execution time
    average_execution_time = sum(execution_times) / len(execution_times)
    print(f"Average Execution Time with {num_shards} shards: {average_execution_time} seconds")

    # Close the connection
    client.close()


Query Result with 1 shards: []
Average Execution Time with 1 shards: 1.4778392016887665 seconds
Query Result with 2 shards: []


KeyboardInterrupt: 

## Data Analyst Queries

### DA1

In [ ]:
{
                        '$lookup': {
                            'from': 'stores', 
                            'localField': 'station_nbr', 
                            'foreignField': 'store_nbr', 
                            'as': 'store_info'
                        }
                    }, {
                        '$unwind': '$store_info'
                    }, {
                        '$lookup': {
                            'from': 'sales', 
                            'localField': 'store_info.store_nbr', 
                            'foreignField': 'store_nbr', 
                            'as': 'sales_info'
                        }
                    }, {
                        '$unwind': '$sales_info'
                    }, {
                        '$lookup': {
                            'from': 'weather', 
                            'localField': 'station_nbr', 
                            'foreignField': 'station_nbr', 
                            'as': 'weather_info'
                        }
                    }, {
                        '$unwind': '$weather_info'
                    }, {
                        '$match': {
                            'weather_info.snowfall': {
                                '$gt': 0
                            }
                        }
                    }, {
                        '$group': {
                            '_id': '$store_info.store_nbr', 
                            'total_units_sold': {
                                '$sum': '$sales_info.units_sold'
                            }
                        }
                    }

In [4]:
from pymongo import MongoClient
import time

# Replace these values with your MongoDB connection details
mongo_host = 'localhost'
mongo_port = 27017
mongo_database = 'Walmart_cloud'
collection_name = 'sales'


# Connect to MongoDB
client = MongoClient(f'mongodb://{mongo_host}:{mongo_port}/')
db = client[mongo_database]
# Access the sharded collection
collection = db[collection_name]

collection.create_index([('stores.store_nbr', 1)])
collection.create_index([('stores.station_nbr', 1)])
collection.create_index([('sales.store_nbr', 1)])
collection.create_index([('weather.station_nbr', 1)])



'weather.station_nbr_1'

In [6]:

# Define the aggregation pipeline
pipeline = [
{
                '$lookup': {
                    'from': 'stores', 
                    'localField': 'station_nbr', 
                    'foreignField': 'store_nbr', 
                    'as': 'store_info'
                }
            }, {
                '$unwind': '$store_info'
            }, {
                '$lookup': {
                    'from': 'sales', 
                    'localField': 'store_info.store_nbr', 
                    'foreignField': 'store_nbr', 
                    'as': 'sales_info'
                }
            }, {
                '$unwind': '$sales_info'
            }, {
                '$lookup': {
                    'from': 'weather', 
                    'localField': 'station_nbr', 
                    'foreignField': 'station_nbr', 
                    'as': 'weather_info'
                }
            }, {
                '$unwind': '$weather_info'
            }, {
                '$match': {
                    'weather_info.snowfall': {
                        '$gt': 0
                    }
                }
            }, {
                '$group': {
                    '_id': '$store_info.store_nbr', 
                    'total_units_sold': {
                        '$sum': '$sales_info.units_sold'
                    }
                }
            }
]


start_time = time.time()
result = list(collection.aggregate(pipeline))
end_time = time.time()
execution_time = end_time - start_time
print("result : ", result)
print("execution time : ", execution_time)


result :  []
execution time :  239.0038285255432


# Streamlit

In [3]:

%run streamlit_app.py


2023-12-16 16:38:09.498 
  command:

    streamlit run streamlit_app.py [ARGUMENTS]


In [23]:
!streamlit run streamlit_app_cyprien.py

^C
